# King County Home Price Predictors - An Analysis

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from statsmodels.formula.api import ols
import statsmodels.api as sm
import scipy.stats as stats

import os
RPSale = pd.read_csv('../../data/raw/EXTR_RPSale.csv')
ResBldg = pd.read_csv('../../data/raw/EXTR_ResBldg.csv')
Parcel = pd.read_csv('../../data/raw/EXTR_Parcel.csv')

# Filtering the Data

### Filtering to The Last 3 Years

In [2]:
Date = RPSale['DocumentDate'].str.split('/', expand=True)
RPSale['MonthSold'] = Date[0].astype('float64')
RPSale['YrSold'] = Date[2].astype('float64')

#Drop original date column
RPSale.drop(columns=['DocumentDate'], axis=1, inplace=True)

In [3]:
#filtering data from 2018, 2019 and 2020 only
years = [2018, 2019, 2020]  
    
properties = RPSale.loc[RPSale['YrSold'].isin(years)]  
    
properties

,ExciseTaxNbr,Major,Minor,SalePrice,RecordingNbr,Volume,Page,PlatNbr,PlatType,PlatLot,...,SaleInstrument,AFForestLand,AFCurrentUseLand,AFNonProfitUse,AFHistoricProperty,SaleReason,PropertyClass,SaleWarning,MonthSold,YrSold
2,2999169,919715,200,192000,20190712001080,,,,,,...,3,N,N,N,N,1,3,,7.0,2019.0
8,3000673,894444,200,185000,20190722001395,,,,,,...,3,N,N,N,N,1,3,,6.0,2019.0
9,3027422,213043,120,560000,20191226000848,,,,,,...,3,N,N,N,N,1,8,,12.0,2019.0
10,3002257,940652,630,435000,20190730001339,,,,,,...,3,N,N,N,N,1,8,,7.0,2019.0
19,3018109,152504,9008,7600000,20191030001615,,,,,,...,3,N,N,N,N,1,2,,10.0,2019.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351056,2997920,302300,320,0,20190705000539,,,,,,...,15,N,N,N,N,1,8,,5.0,2019.0
351057,3028691,277110,4239,955000,20200103000685,,,,,,...,3,N,N,N,N,1,8,,12.0,2019.0
351058,2934041,388580,3260,1375000,20180604000929,,,,,,...,3,N,N,N,N,1,8,,5.0,2018.0
351062,2935450,219331,270,850000,20180612000762,,,,,,...,3,N,N,N,N,1,8,,6.0,2018.0


In [4]:
properties['YrSold'].value_counts()

2019.0    61351
2018.0    61170
2020.0    20656
Name: YrSold, dtype: int64

In [5]:
properties = properties.rename(columns=str.lower)

In [6]:
properties.head()

,excisetaxnbr,major,minor,saleprice,recordingnbr,volume,page,platnbr,plattype,platlot,...,saleinstrument,afforestland,afcurrentuseland,afnonprofituse,afhistoricproperty,salereason,propertyclass,salewarning,monthsold,yrsold
2,2999169,919715,200,192000,20190712001080,,,,,,...,3,N,N,N,N,1,3,,7.0,2019.0
8,3000673,894444,200,185000,20190722001395,,,,,,...,3,N,N,N,N,1,3,,6.0,2019.0
9,3027422,213043,120,560000,20191226000848,,,,,,...,3,N,N,N,N,1,8,,12.0,2019.0
10,3002257,940652,630,435000,20190730001339,,,,,,...,3,N,N,N,N,1,8,,7.0,2019.0
19,3018109,152504,9008,7600000,20191030001615,,,,,,...,3,N,N,N,N,1,2,,10.0,2019.0


### Padding the 'major' and 'minor' columns and merging them together:

In [7]:
properties['major'] = properties['major'].astype(str)
properties['minor'] = properties['minor'].astype(str)

def major_padded(row):
    if len(row) == 6:
        return row
    elif len(row) == 5:
        return '0'+row
    elif len(row) == 4:
        return '00'+row
    elif len(row) == 3:
        return '000'+row
    elif len(row) == 2:
        return '0000'+row
    elif len(row) == 1:
        return '00000'+row
    else:
        return '000000'

def minor_padded(row):
    if len(row) == 4:
        return row
    elif len(row) == 3: 
        return '0'+row
    elif len(row) == 2:
        return '00'+row
    elif len(row) == 1:
        return '000'+row
    else:
        return '0000'
    
properties['major'] = properties.major.apply(major_padded)
properties['minor'] = properties.minor.apply(minor_padded)
properties['major_minor'] = properties['major']+properties['minor']

In [8]:
#Dropping the old columns
to_drop = ['major','minor']
properties.drop(to_drop, axis=1, inplace=True)

In [9]:
#Moving the 'major_minor' column to the front
col_name = 'major_minor'
last_col = properties.pop(col_name)
properties.insert(1, col_name, last_col)

### Narrowing the property type
According to our LookUp file, the residental values are coded as follows:

**11** - Household, single family units

**12** - Multiple family residence (Residential, 2-4 units)

**14** - Residential condominiums

**18** - All other residential not elsewhere coded

In [10]:
properties = properties[properties['propertytype'].isin([11, 12, 14, 18])]
properties

,excisetaxnbr,major_minor,saleprice,recordingnbr,volume,page,platnbr,plattype,platlot,platblock,...,saleinstrument,afforestland,afcurrentuseland,afnonprofituse,afhistoricproperty,salereason,propertyclass,salewarning,monthsold,yrsold
9,3027422,2130430120,560000,20191226000848,,,,,,,...,3,N,N,N,N,1,8,,12.0,2019.0
10,3002257,9406520630,435000,20190730001339,,,,,,,...,3,N,N,N,N,1,8,,7.0,2019.0
22,3015516,7797900030,0,20191016000009,,,,,,,...,3,N,N,N,N,1,8,,10.0,2019.0
24,2963345,7574800320,0,20181126001127,,,,,,,...,15,N,N,N,N,1,3,,11.0,2018.0
27,3033993,9264902060,370000,20200214001331,,,,,,,...,2,N,N,N,N,1,8,,2.0,2020.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351043,3045715,4127000898,619000,20200505001286,,,,,,,...,2,N,N,N,N,1,8,,4.0,2020.0
351047,2949086,3597000405,0,20180824000032,,,,,,,...,15,N,N,N,N,1,8,,7.0,2018.0
351056,2997920,3023000320,0,20190705000539,,,,,,,...,15,N,N,N,N,1,8,,5.0,2019.0
351062,2935450,2193310270,850000,20180612000762,,,,,,,...,3,N,N,N,N,1,8,,6.0,2018.0


### Only using sale prices over 0:

In [11]:
properties = properties[properties['saleprice'] > 0]
properties

,excisetaxnbr,major_minor,saleprice,recordingnbr,volume,page,platnbr,plattype,platlot,platblock,...,saleinstrument,afforestland,afcurrentuseland,afnonprofituse,afhistoricproperty,salereason,propertyclass,salewarning,monthsold,yrsold
9,3027422,2130430120,560000,20191226000848,,,,,,,...,3,N,N,N,N,1,8,,12.0,2019.0
10,3002257,9406520630,435000,20190730001339,,,,,,,...,3,N,N,N,N,1,8,,7.0,2019.0
27,3033993,9264902060,370000,20200214001331,,,,,,,...,2,N,N,N,N,1,8,,2.0,2020.0
42,3043271,5703500010,575000,20200417000249,,,,,,,...,2,N,N,N,N,1,8,,4.0,2020.0
50,2937935,8731780100,185000,20180625000560,,,,,,,...,3,N,N,N,N,1,3,,6.0,2018.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351038,3025068,6150200505,745000,20191212000602,,,,,,,...,3,N,N,N,N,1,8,,11.0,2019.0
351040,2986698,0333100255,6500000,20190508000847,,,,,,,...,3,N,N,N,N,1,8,,4.0,2019.0
351041,3038910,0267700310,455000,20200317001112,,,,,,,...,2,N,N,N,N,1,3,,3.0,2020.0
351043,3045715,4127000898,619000,20200505001286,,,,,,,...,2,N,N,N,N,1,8,,4.0,2020.0


In [12]:
properties['saleprice'].describe()

count    4.776000e+04
mean     7.446527e+05
std      6.547413e+05
min      5.000000e+00
25%      4.250000e+05
50%      6.180000e+05
75%      8.680000e+05
max      5.300000e+07
Name: saleprice, dtype: float64

### Filtering out based on principal use:

In [13]:
properties['principaluse'].unique()

array([6, 2, 7, 0], dtype=int64)

According to our lookup, 

**6** - Residential

**2** - Condominium

**7** - Commercial

I will only focus on residential and condo properties.

In [14]:
properties = properties[properties['principaluse'].isin([6,2])]
properties

,excisetaxnbr,major_minor,saleprice,recordingnbr,volume,page,platnbr,plattype,platlot,platblock,...,saleinstrument,afforestland,afcurrentuseland,afnonprofituse,afhistoricproperty,salereason,propertyclass,salewarning,monthsold,yrsold
9,3027422,2130430120,560000,20191226000848,,,,,,,...,3,N,N,N,N,1,8,,12.0,2019.0
10,3002257,9406520630,435000,20190730001339,,,,,,,...,3,N,N,N,N,1,8,,7.0,2019.0
27,3033993,9264902060,370000,20200214001331,,,,,,,...,2,N,N,N,N,1,8,,2.0,2020.0
42,3043271,5703500010,575000,20200417000249,,,,,,,...,2,N,N,N,N,1,8,,4.0,2020.0
50,2937935,8731780100,185000,20180625000560,,,,,,,...,3,N,N,N,N,1,3,,6.0,2018.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351038,3025068,6150200505,745000,20191212000602,,,,,,,...,3,N,N,N,N,1,8,,11.0,2019.0
351040,2986698,0333100255,6500000,20190508000847,,,,,,,...,3,N,N,N,N,1,8,,4.0,2019.0
351041,3038910,0267700310,455000,20200317001112,,,,,,,...,2,N,N,N,N,1,3,,3.0,2020.0
351043,3045715,4127000898,619000,20200505001286,,,,,,,...,2,N,N,N,N,1,8,,4.0,2020.0


### Checking for 'propertyclass'

In [15]:
properties['propertyclass'].unique()

array([8, 3, 7, 9], dtype=int64)

The follwing values and their definitions are found in our lookup file:

**7** - Res-Land only (I will remove this to only display residential property)

I will keep:

**8** - Residential- Improved Property

**3** - Condominium

**9** - Res/Condo mobile

In [16]:
properties = properties[properties['propertyclass'].isin([8,3,9])]
properties.head()

,excisetaxnbr,major_minor,saleprice,recordingnbr,volume,page,platnbr,plattype,platlot,platblock,...,saleinstrument,afforestland,afcurrentuseland,afnonprofituse,afhistoricproperty,salereason,propertyclass,salewarning,monthsold,yrsold
9,3027422,2130430120,560000,20191226000848,,,,,,,...,3,N,N,N,N,1,8,,12.0,2019.0
10,3002257,9406520630,435000,20190730001339,,,,,,,...,3,N,N,N,N,1,8,,7.0,2019.0
27,3033993,9264902060,370000,20200214001331,,,,,,,...,2,N,N,N,N,1,8,,2.0,2020.0
42,3043271,5703500010,575000,20200417000249,,,,,,,...,2,N,N,N,N,1,8,,4.0,2020.0
50,2937935,8731780100,185000,20180625000560,,,,,,,...,3,N,N,N,N,1,3,,6.0,2018.0


### Checking for duplicates based on major_minor numbers

In [17]:
len(properties) - len(properties.major_minor.unique())

1585

I will check to see if there are any trends in the duplicates, since it is possible a home could have been sold more than once during the set timeframe.

In [18]:
properties[properties['major_minor'].isin(['8843900445','8847500000'])]

,excisetaxnbr,major_minor,saleprice,recordingnbr,volume,page,platnbr,plattype,platlot,platblock,...,saleinstrument,afforestland,afcurrentuseland,afnonprofituse,afhistoricproperty,salereason,propertyclass,salewarning,monthsold,yrsold
41899,2975737,8843900445,815000,20190228001472,,,,,,,...,3,N,N,N,N,1,8,,2.0,2019.0
149957,2975141,8843900445,699900,20190226001189,,,,,,,...,3,N,N,N,N,1,8,,2.0,2019.0
162622,2993692,8847500000,764990,20190614000861,,,,,,,...,3,N,N,N,N,1,3,,6.0,2019.0
197942,2976965,8843900445,706900,20190311000874,,,,,,,...,3,N,N,N,N,1,8,,3.0,2019.0
210949,3020478,8847500000,698654,20191114000503,,,,,,,...,3,N,N,N,N,1,3,,11.0,2019.0
292126,2980974,8843900445,799900,20190405000497,,,,,,,...,3,N,N,N,N,1,8,,3.0,2019.0
294166,2975973,8843900445,782900,20190301000951,,,,,,,...,3,N,N,N,N,1,8,,2.0,2019.0
312455,2975726,8843900445,776900,20190228001436,,,,,,,...,3,N,N,N,N,1,8,,2.0,2019.0
332992,2976878,8847500000,784684,20190311000330,,,,,,,...,3,N,N,N,N,1,3,43,3.0,2019.0


In [19]:
#Dropping the duplicates
properties.drop_duplicates(subset='major_minor', keep='last', inplace=True)

<ipython-input-19-6281fb0af3f5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  properties.drop_duplicates(subset='major_minor', keep='last', inplace=True)


In [20]:
properties.head()

,excisetaxnbr,major_minor,saleprice,recordingnbr,volume,page,platnbr,plattype,platlot,platblock,...,saleinstrument,afforestland,afcurrentuseland,afnonprofituse,afhistoricproperty,salereason,propertyclass,salewarning,monthsold,yrsold
9,3027422,2130430120,560000,20191226000848,,,,,,,...,3,N,N,N,N,1,8,,12.0,2019.0
10,3002257,9406520630,435000,20190730001339,,,,,,,...,3,N,N,N,N,1,8,,7.0,2019.0
27,3033993,9264902060,370000,20200214001331,,,,,,,...,2,N,N,N,N,1,8,,2.0,2020.0
50,2937935,8731780100,185000,20180625000560,,,,,,,...,3,N,N,N,N,1,3,,6.0,2018.0
55,2926991,2211700075,1425000,20180427001349,,,,,,,...,3,N,N,N,N,1,8,,4.0,2018.0


In [21]:
properties.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45203 entries, 9 to 351062
Data columns (total 24 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   excisetaxnbr        45203 non-null  int64  
 1   major_minor         45203 non-null  object 
 2   saleprice           45203 non-null  int64  
 3   recordingnbr        45203 non-null  object 
 4   volume              45203 non-null  object 
 5   page                45203 non-null  object 
 6   platnbr             45203 non-null  object 
 7   plattype            45203 non-null  object 
 8   platlot             45203 non-null  object 
 9   platblock           45203 non-null  object 
 10  sellername          45203 non-null  object 
 11  buyername           45203 non-null  object 
 12  propertytype        45203 non-null  int64  
 13  principaluse        45203 non-null  int64  
 14  saleinstrument      45203 non-null  int64  
 15  afforestland        45203 non-null  object 
 16  afc

In [22]:
to_drop = ['volume', 'page', 'platnbr', 'plattype', 'platlot', 'platblock', 'sellername', 'buyername']
properties.drop(to_drop, axis=1, inplace=True)

C:\Users\15164\anaconda3\envs\learn-env\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [23]:
properties.head()

,excisetaxnbr,major_minor,saleprice,recordingnbr,propertytype,principaluse,saleinstrument,afforestland,afcurrentuseland,afnonprofituse,afhistoricproperty,salereason,propertyclass,salewarning,monthsold,yrsold
9,3027422,2130430120,560000,20191226000848,11,6,3,N,N,N,N,1,8,,12.0,2019.0
10,3002257,9406520630,435000,20190730001339,11,6,3,N,N,N,N,1,8,,7.0,2019.0
27,3033993,9264902060,370000,20200214001331,11,6,2,N,N,N,N,1,8,,2.0,2020.0
50,2937935,8731780100,185000,20180625000560,14,2,3,N,N,N,N,1,3,,6.0,2018.0
55,2926991,2211700075,1425000,20180427001349,11,6,3,N,N,N,N,1,8,,4.0,2018.0


Now I'll repeat everything I did for "properties" and apply it to residential buildings.

In [24]:
ResBldg.rename(columns=str.lower, inplace=True)
ResBldg['major'] = ResBldg['major'].astype(str)
ResBldg['minor'] = ResBldg['minor'].astype(str)

ResBldg['major'] = ResBldg['major'].apply(major_padded)
ResBldg['minor'] = ResBldg['minor'].apply(minor_padded)

ResBldg['major_minor'] = ResBldg['major']+ResBldg['minor']

col_name = 'major_minor'
last_col = ResBldg.pop(col_name)
ResBldg.insert(0, col_name, last_col)

ResBldg.head(2)

,major_minor,major,minor,bldgnbr,nbrlivingunits,address,buildingnumber,fraction,directionprefix,streetname,...,fpmultistory,fpfreestanding,fpadditional,yrbuilt,yrrenovated,pcntcomplete,obsolescence,pcntnetcondition,condition,addnlcost
0,0098000720,009800,0720,1,1,27719 SE 26TH WAY 98075,27719,,SE,26TH,...,0,0,0,2001,0,0,0,0,3,0
1,0098020140,009802,0140,1,1,2829 277TH TER SE 98075,2829,,,277TH,...,0,0,0,2004,0,0,0,0,3,0


In [25]:
sum(ResBldg.minor == '0000')

0

Since residential buildings don't have condominiums (**Minor=0000**), I will be forced to drop more data.

In [26]:
ResBldg.loc[1]

major_minor                           0098020140
major                                     009802
minor                                       0140
bldgnbr                                        1
nbrlivingunits                                 1
address               2829   277TH TER SE  98075
buildingnumber                             2829 
fraction                                        
directionprefix                                 
streetname             277TH                    
streettype                                TER   
directionsuffix                               SE
zipcode                                    98075
stories                                        2
bldggrade                                     10
bldggradevar                                   0
sqft1stfloor                                1610
sqfthalffloor                                  0
sqft2ndfloor                                1400
sqftupperfloor                                 0
sqftunfinfull       

In [27]:
ResBldg = ResBldg.dropna(how='any',axis=0)
ResBldg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 154594 entries, 0 to 181506
Data columns (total 51 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   major_minor         154594 non-null  object 
 1   major               154594 non-null  object 
 2   minor               154594 non-null  object 
 3   bldgnbr             154594 non-null  int64  
 4   nbrlivingunits      154594 non-null  int64  
 5   address             154594 non-null  object 
 6   buildingnumber      154594 non-null  object 
 7   fraction            154594 non-null  object 
 8   directionprefix     154594 non-null  object 
 9   streetname          154594 non-null  object 
 10  streettype          154594 non-null  object 
 11  directionsuffix     154594 non-null  object 
 12  zipcode             154594 non-null  object 
 13  stories             154594 non-null  float64
 14  bldggrade           154594 non-null  int64  
 15  bldggradevar        154594 non-nul

### Converting 'zipcode' to int

In [28]:
ResBldg['zipcode'] = ResBldg['zipcode'].str.split('-').str[0]
ResBldg['zipcode'].value_counts()

98042    4661
98023    4177
98115    4175
98038    3828
98058    3733
         ... 
98          1
90108       1
89045       1
98101       1
98085       1
Name: zipcode, Length: 122, dtype: int64

In [29]:
ResBldg['zipcode'] = ResBldg['zipcode'].astype(str).str.replace(',', '')
ResBldg['zipcode'] = ResBldg['zipcode'].astype(str).str.replace(' ', '')
ResBldg['zipcode'] = ResBldg['zipcode'].astype(str).str.replace('.', '')

In [30]:
df = ResBldg[ResBldg['zipcode'].apply(lambda x: not x.isnumeric())]
df

,major_minor,major,minor,bldgnbr,nbrlivingunits,address,buildingnumber,fraction,directionprefix,streetname,...,fpmultistory,fpfreestanding,fpadditional,yrbuilt,yrrenovated,pcntcomplete,obsolescence,pcntnetcondition,condition,addnlcost
71823,3126069112,312606,9112,1,1,11026 AVONDALE RD NE,11026,,,AVONDALE,...,0,0,0,2015,0,0,0,0,3,8000
173514,9828702305,982870,2305,1,1,111 D 26TH AVE E A on plans,111,D,,26TH,...,0,0,0,2019,0,60,0,0,3,0


In [31]:
#Dropping non-numeric zipcode entries
ResBldg = ResBldg.drop(index=[71823,173514])

In [32]:
ResBldg['zipcode'].value_counts()

98042    4661
98023    4177
98115    4175
98038    3828
98058    3756
         ... 
98085       1
89045       1
90333       1
98083       1
98026       1
Name: zipcode, Length: 119, dtype: int64

In [33]:
ResBldg['zipcode']= ResBldg['zipcode'].astype(np.int64)

In [34]:
prop_res = properties.merge(ResBldg, on='major_minor', suffixes=("", " "))
prop_res.describe()

,excisetaxnbr,saleprice,propertytype,principaluse,saleinstrument,salereason,propertyclass,monthsold,yrsold,bldgnbr,...,fpmultistory,fpfreestanding,fpadditional,yrbuilt,yrrenovated,pcntcomplete,obsolescence,pcntnetcondition,condition,addnlcost
count,3.392800e+04,3.392800e+04,33928.000000,33928.0,33928.000000,33928.000000,33928.0,33928.000000,33928.000000,33928.000000,...,33928.000000,33928.000000,33928.000000,33928.000000,33928.000000,33928.000000,33928.000000,33928.000000,33928.000000,33928.000000
mean,2.987995e+06,7.676240e+05,11.024228,6.0,3.073450,1.014914,8.0,6.396781,2018.826839,1.008872,...,0.337921,0.078961,0.198744,1968.021752,110.160133,0.173898,0.110410,0.006425,3.536401,597.927995
std,3.762621e+04,5.874356e+05,0.280116,0.0,2.185849,0.501786,0.0,3.174764,0.708859,0.113924,...,0.512201,0.280399,0.418317,28.385749,456.266456,3.498123,2.103985,0.457740,0.718633,2286.108745
min,2.909403e+06,5.000000e+00,11.000000,6.0,2.000000,1.000000,8.0,1.000000,2018.000000,1.000000,...,0.000000,0.000000,0.000000,1900.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
25%,2.956618e+06,4.500000e+05,11.000000,6.0,3.000000,1.000000,8.0,4.000000,2018.000000,1.000000,...,0.000000,0.000000,0.000000,1950.000000,0.000000,0.000000,0.000000,0.000000,3.000000,0.000000
50%,2.992006e+06,6.450000e+05,11.000000,6.0,3.000000,1.000000,8.0,6.000000,2019.000000,1.000000,...,0.000000,0.000000,0.000000,1969.000000,0.000000,0.000000,0.000000,0.000000,3.000000,0.000000
75%,3.019295e+06,8.850000e+05,11.000000,6.0,3.000000,1.000000,8.0,9.000000,2019.000000,1.000000,...,1.000000,0.000000,0.000000,1990.000000,0.000000,0.000000,0.000000,0.000000,4.000000,0.000000
max,3.050093e+06,2.675000e+07,18.000000,6.0,27.000000,18.000000,8.0,12.000000,2020.000000,7.000000,...,5.000000,3.000000,3.000000,2020.000000,2020.000000,100.000000,99.000000,43.000000,5.000000,50000.000000


### Converting the 'yrrenovated' column to a binary
Next, the 'yrrenovated' column has a large amount zero values (denoting that a house hasn't been renovated). Rather than dropping the data, I will make this a binary column whether a house was sold within 10 years of a renovation or not, or built within the last five years. 

First, I will make a new column 'age' with the properties' ages as variables to get a better sense of how old a home is.

In [35]:
#Convert YrBuilt to age by subtracting the build year from the most recent year in the in dataframe
prop_res['age'] = 2020 - prop_res.yrbuilt

Now I will convert the 'yrrenovated' column to a binary column.

In [36]:
prop_res['yrrenovated'].value_counts()

0       32059
2018      173
2019      122
2017       66
2003       58
        ...  
1935        1
1949        1
1944        1
1952        1
1936        1
Name: yrrenovated, Length: 80, dtype: int64

In [37]:
prop_res['yrrenovated'].fillna(0.0, inplace=True)

In [38]:
prop_res['renovated'] = prop_res['yrsold'] - prop_res['yrrenovated']

In [39]:
#Replacing values less than 10 with 1, and values over 10 with 0
Renovated = prop_res.renovated.values
Age = prop_res.age.values
values = np.where(Renovated <= 10, 1, 0)
prop_res['renovated'] = np.where(Age <= 5, 1, values)

Now I'll check again for duplicates:

In [40]:
prop_res.major_minor.value_counts()

0809003030    5
6373000170    5
7129303601    4
2822059059    4
6196600360    4
             ..
8651402910    1
3276980010    1
1822059374    1
3432501415    1
3352400905    1
Name: major_minor, Length: 33671, dtype: int64

In [41]:
prop_res[prop_res['major_minor'].isin(['0809003030', '6373000170', '2822059059'])]

,excisetaxnbr,major_minor,saleprice,recordingnbr,propertytype,principaluse,saleinstrument,afforestland,afcurrentuseland,afnonprofituse,...,fpadditional,yrbuilt,yrrenovated,pcntcomplete,obsolescence,pcntnetcondition,condition,addnlcost,age,renovated
1886,3048606,0809003030,1176000,20200527001133,11,6,3,N,N,N,...,0,2019,0,40,0,0,3,0,1,1
1887,3048606,0809003030,1176000,20200527001133,11,6,3,N,N,N,...,0,2019,0,40,0,0,3,0,1,1
1888,3048606,0809003030,1176000,20200527001133,11,6,3,N,N,N,...,0,2019,0,40,0,0,3,0,1,1
1889,3048606,0809003030,1176000,20200527001133,11,6,3,N,N,N,...,0,2019,0,40,0,0,3,0,1,1
1890,3048606,0809003030,1176000,20200527001133,11,6,3,N,N,N,...,0,2019,0,40,0,0,3,0,1,1
24272,2921369,6373000170,1500000,20180328000779,11,6,3,N,N,N,...,0,2019,0,60,0,0,3,0,1,1
24273,2921369,6373000170,1500000,20180328000779,11,6,3,N,N,N,...,0,2019,0,60,0,0,3,0,1,1
24274,2921369,6373000170,1500000,20180328000779,11,6,3,N,N,N,...,0,2019,0,60,0,0,3,0,1,1
24275,2921369,6373000170,1500000,20180328000779,11,6,3,N,N,N,...,0,2019,0,60,0,0,3,0,1,1
24276,2921369,6373000170,1500000,20180328000779,11,6,3,N,N,N,...,0,2019,0,60,0,0,3,0,1,1


In [42]:
prop_res.drop_duplicates(subset='major_minor', inplace=True)
len(prop_res)

33671

We are now left with 33,671 entries in our data frame.

### Now for our last file, 'Parcel':

In [43]:
Parcel.rename(columns=str.lower, inplace=True)
Parcel['major'] = Parcel['major'].astype(str)
Parcel['minor'] = Parcel['minor'].astype(str)
Parcel['major'] = Parcel['major'].apply(major_padded)
Parcel['minor'] = Parcel['minor'].apply(minor_padded)
Parcel['major_minor'] = Parcel['major']+Parcel['minor']
last_col = Parcel.pop(col_name)
Parcel.insert(0, col_name, last_col)

In [44]:
prop_res = prop_res.merge(Parcel, on='major_minor', suffixes=("", " "))

In [45]:
prop_res.shape

(33671, 150)

In [46]:
prop_res['proptype'].unique()

array(['R'], dtype=object)

Thankfully, we haven't lost any more entries *and* our property type column only contains 'R' (residential) values.

In [47]:
prop_res.describe()

,excisetaxnbr,saleprice,propertytype,principaluse,saleinstrument,salereason,propertyclass,monthsold,yrsold,bldgnbr,...,wfntpoorquality,wfntrestrictedaccess,tidelandshoreland,lotdepthfactor,trafficnoise,airportnoise,nbrbldgsites,contamination,historicsite,currentusedesignation
count,3.367100e+04,3.367100e+04,33671.000000,33671.0,33671.00000,33671.000000,33671.0,33671.000000,33671.000000,33671.000000,...,33671.000000,33671.000000,33671.000000,33671.000000,33671.000000,33671.0,33671.00000,33671.0,33671.000000,33671.000000
mean,2.987987e+06,7.654065e+05,11.023908,6.0,3.07110,1.015028,8.0,6.402958,2018.826141,1.003772,...,0.000713,0.005554,0.015117,0.049004,0.207270,0.0,0.00689,0.0,0.001277,0.002762
std,3.760185e+04,5.794149e+05,0.280319,0.0,2.17568,0.503696,0.0,3.172576,0.708517,0.071995,...,0.026689,0.108990,0.172277,2.031641,0.554123,0.0,0.14849,0.0,0.060917,0.087662
min,2.909403e+06,5.000000e+00,11.000000,6.0,2.00000,1.000000,8.0,1.000000,2018.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.0,0.000000,0.000000
25%,2.956624e+06,4.500000e+05,11.000000,6.0,3.00000,1.000000,8.0,4.000000,2018.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.0,0.000000,0.000000
50%,2.992018e+06,6.450000e+05,11.000000,6.0,3.00000,1.000000,8.0,6.000000,2019.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.0,0.000000,0.000000
75%,3.018590e+06,8.840000e+05,11.000000,6.0,3.00000,1.000000,8.0,9.000000,2019.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.0,0.000000,0.000000
max,3.050093e+06,2.675000e+07,18.000000,6.0,27.00000,18.000000,8.0,12.000000,2020.000000,7.000000,...,1.000000,3.000000,3.000000,109.000000,3.000000,0.0,16.00000,0.0,3.000000,3.000000


**I will select features of a residential property that would have a bearing on its saleprice:**

In [48]:
property_data = prop_res.loc[:, ['saleprice', 'sqfttotliving', 'yrbuilt', 'yrrenovated', 'bedrooms',
                      'zipcode', 'sqfttotbasement', 'sqftfinbasement', 'sqftopenporch',
                      'sqftenclosedporch', 'sqftdeck','heatsystem', 'heatsource', 'bathhalfcount', 
                      'bath3qtrcount', 'bathfullcount','condition', 'viewutilization', 'sqftgarageattached',
                      'daylightbasement', 'finbasementgrade', 'hbuasifvacant', 'inadequateparking', 'township',
                      'mtrainier', 'olympics', 'cascades', 'territorial', 'seattleskyline', 'pugetsound',
                      'lakewashington', 'lakesammamish', 'smalllakerivercreek', 'otherview', 'wfntlocation',
                      'trafficnoise', 'airportnoise', 'powerlines', 'othernuisances', 'adjacentgreenbelt']]

In [49]:
property_data.head()

,saleprice,sqfttotliving,yrbuilt,yrrenovated,bedrooms,zipcode,sqfttotbasement,sqftfinbasement,sqftopenporch,sqftenclosedporch,...,lakewashington,lakesammamish,smalllakerivercreek,otherview,wfntlocation,trafficnoise,airportnoise,powerlines,othernuisances,adjacentgreenbelt
0,560000,1930,1989,0,3,98019,0,0,180,0,...,0,0,0,0,0,0,0,N,N,N
1,435000,1975,1994,0,3,98038,0,0,45,0,...,0,0,0,0,0,0,0,N,N,N
2,370000,1750,1979,0,3,98023,1130,460,0,0,...,0,0,0,0,0,0,0,N,N,N
3,1425000,2850,1985,0,4,98006,0,0,190,0,...,0,0,0,0,0,0,0,N,N,N
4,1085000,1870,2008,0,3,98102,790,790,0,0,...,0,0,0,0,0,0,0,N,N,N


In [50]:
cleaned_data = property_data.to_csv('cleaned_data.csv')
property_data = prop_res.to_csv('property_data.csv')

In [1]:
kc_data = pd.read_csv('../../data/processed/cleaned_data.csv')
kc_data.head()

NameError: name 'pd' is not defined